<a href="https://colab.research.google.com/github/wabinyai/Mapping-with-Folium-/blob/main/%5B11_03_2022%5D_Kampala_CHOROPLETH_N2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install geopandas

     |████████████████████████████████| 1.0 MB 15.3 MB/s 
     |████████████████████████████████| 6.3 MB 39.3 MB/s 
     |████████████████████████████████| 16.7 MB 38.4 MB/s 


In [ ]:
pip install iteration_utilities

# Importing the necessary libraries

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import branca.colormap as cm
import matplotlib.pyplot as plt
from iteration_utilities import duplicates
from google.colab import files
from folium.plugins import FastMarkerCluster

# Upload the json files

In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
  print ('Json dataset'.format(
      name=fn, length=len(uploaded[fn])
  ))

Saving Kla.json to Kla (1).json
Avacado dataset


In [ ]:
fname = fn
kla_geo = gpd.read_file(fname)
kla_geo.head()

,id,UBOS_PN_13,UBOS_PN_06,UBOS_SN_13,UBOS_SN_06,UBOS_DN_12,Male2002,Female2002,Total2002,Male2012,...,Total2012,Total2013,PopDensity,Shape_Leng,Shape_Area,created_us,created_da,last_edite,last_edi_1,geometry
0,None,Ggaba,Ggaba,MAKINDYE,Makindye,KAMPALA,9605,10607,20212,13900,...,29400,30408.337430,4391.119070,18522.619313,6.695332e+06,None,None,None,None,"MULTIPOLYGON (((32.65672 0.27464, 32.65677 0.2..."
1,None,Luzira,Luzira,NAKAWA,Nakawa,KAMPALA,7355,8071,15426,10600,...,22400,23182.303210,6512.657693,14850.056790,3.439456e+06,None,None,None,None,"MULTIPOLYGON (((32.66630 0.29248, 32.66631 0.2..."
2,None,Rubaga,Rubaga,RUBAGA,Rubaga,KAMPALA,9284,11423,20707,13600,...,30300,31397.166310,9072.882488,7845.823752,3.339622e+06,None,None,None,None,"POLYGON ((32.55831 0.30785, 32.55848 0.30739, ..."
3,None,Civic Centre,Civic Centre,CENTRAL,Central,KAMPALA,645,562,1207,1000,...,1900,1935.834785,1572.890739,5666.401709,1.207967e+06,None,None,None,None,"POLYGON ((32.59345 0.31752, 32.59342 0.31750, ..."
4,None,Makerere University,Makerere University,KAWEMPE,Kawempe,KAMPALA,2389,2262,4651,3400,...,6700,6976.106971,5552.764196,4637.684949,1.206606e+06,None,None,None,None,"POLYGON ((32.57134 0.34083, 32.57147 0.34024, ..."


# Upload Garage GPS

In [ ]:
uploaded = files.upload()
for fn1 in uploaded.keys():
  print ('Garage gps dataset'.format(
      name=fn1, length=len(uploaded[fn1])
  ))

Saving Garages_gps_location.csv to Garages_gps_location (1).csv
Avacado dataset


In [ ]:
kla_garage = pd.read_csv(fn1)
kla_garage.head()

,parish,latitude,longitude,Name,Vehicle
0,Wabigalo,0.313991,32.604819,a,12
1,Wabigalo,0.311347,32.604967,b,45
2,Wabigalo,0.311917,32.605092,c,63
3,Kibuli,0.308583,32.597411,d,74
4,Kibuli,0.308292,32.597281,e,25


In [ ]:
kla_geo=kla_geo[['UBOS_PN_13','geometry']]

kla_geo.shape

(96, 2)

In [ ]:

kla_garage = kla_garage.rename(columns={'parish': 'UBOS_PN_13'})
kla_garage.head()

,UBOS_PN_13,latitude,longitude,Name,Vehicle
0,Wabigalo,0.313991,32.604819,a,12
1,Wabigalo,0.311347,32.604967,b,45
2,Wabigalo,0.311917,32.605092,c,63
3,Kibuli,0.308583,32.597411,d,74
4,Kibuli,0.308292,32.597281,e,25


In [ ]:
#sorting and removing duplicates from geo data frame
kla_geo.sort_values("UBOS_PN_13", inplace = True)
kla_geo1= kla_geo.drop_duplicates(subset ="UBOS_PN_13", 
                     keep = False, inplace = False)

In [ ]:
# Merge and create a geopandas dataframe
klageo_gdf=kla_geo1.merge(kla_garage,on="UBOS_PN_13")


In [ ]:
# Checking if all the columns are populated as required
klageo_gdf.head()

,UBOS_PN_13,geometry,latitude,longitude,Name,Vehicle
0,Kibuli,"POLYGON ((32.59830 0.31751, 32.59832 0.31724, ...",0.308583,32.597411,d,74
1,Kibuli,"POLYGON ((32.59830 0.31751, 32.59832 0.31724, ...",0.308292,32.597281,e,25
2,Wabigalo,"POLYGON ((32.60031 0.30918, 32.60029 0.30924, ...",0.313991,32.604819,a,12
3,Wabigalo,"POLYGON ((32.60031 0.30918, 32.60029 0.30924, ...",0.311347,32.604967,b,45
4,Wabigalo,"POLYGON ((32.60031 0.30918, 32.60029 0.30924, ...",0.311917,32.605092,c,63


In [ ]:
# Checking if all the columns are populated as required
klageo_gdf.shape

(10, 6)

# Creating Choropleth Map

In [ ]:
# Assigning centroids to the map
x_map=klageo_gdf.centroid.x.mean()
y_map=klageo_gdf.centroid.y.mean()
print(x_map,y_map)

32.60094303125417 0.31369923799865007


In [ ]:
Kla_inc_map = folium.Map(location=[y_map, x_map], zoom_start=12,tiles=None)
#folium.TileLayer('Stamen Terrain',name="Light Map",control=False).add_to(Kla_inc_map)
folium.TileLayer('Openstreetmap').add_to(Kla_inc_map)
folium.LayerControl
Kla_inc_map  

In [ ]:
Kla_inc_map

In [ ]:

#df = df.to_crs(epsg = 4326)
klageo_gdf.crs = "epsg:4326"


In [ ]:
# Creating choropleth map

Kla_inc_map.choropleth(geo_data=klageo_gdf,
             name='Choropleth',         
             data=klageo_gdf,
             columns=['UBOS_PN_13','Vehicle'], 
             key_on='feature.properties.UBOS_PN_13', 
             fill_color='YlGn',
             fill_opacity=0.2, 
             line_opacity=0.8,
          #   threshold_scale=scale,     
             legend_name='Pollution (%)', 
             smooth_factor=0,     
             highlight=True
                 )

Kla_inc_map    

/usr/local/lib/python3.7/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [ ]:
# Adding informative lables to the map using geoJson

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.5, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}





In [ ]:
 #klageo_gdf.head()

In [ ]:

SuburbName = folium.features.GeoJson(
    klageo_gdf,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['UBOS_PN_13'
 #               ,'UBOS_PN_13'
 #               ,'UBOS_PN_13'
 #               ,'  Yearly average taxable income of Female   '
 #               ,'  Yearly average taxable income of Male   '
               ],
        aliases=['Parish :: '
 #               ,'Division :: '
 #               ,'CITY: '
 #               ,'  Yearly average taxable income of Female   : '
 #               ,'  Yearly average taxable income of Male   : ' 
                 ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
Kla_inc_map.add_child(SuburbName)
Kla_inc_map.keep_in_front(SuburbName)

# Adding folium layer to the map
folium.LayerControl().add_to(Kla_inc_map)


In [ ]:
Kla_inc_map

In [ ]:

SuburbName = folium.features.GeoJson(
    klageo_gdf,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['UBOS_PN_13'
 #               ,'UBOS_PN_13'
 #               ,'UBOS_PN_13'
 #               ,'  Yearly average taxable income of Female   '
 #               ,'  Yearly average taxable income of Male   '
               ],
        aliases=['Parish :: '
 #               ,'Division :: '
 #               ,'CITY: '
 #               ,'  Yearly average taxable income of Female   : '
 #               ,'  Yearly average taxable income of Male   : ' 
                 ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
Kla_inc_map.add_child(SuburbName)
Kla_inc_map.keep_in_front(SuburbName)

# Adding folium layer to the map
folium.LayerControl().add_to(Kla_inc_map)


Kla_inc_map


In [ ]:
# Creating choropleth map
m = folium.Map(location=[y_map, x_map], zoom_start=12,tiles=None)
#folium.TileLayer('Stamen Terrain',name="Light Map",control=False).add_to(Kla_inc_map)
folium.TileLayer('Openstreetmap').add_to(m)
folium.LayerControl



m.choropleth(geo_data=klageo_gdf,
             name='Choropleth',         
             data=klageo_gdf,
             columns=['UBOS_PN_13','Vehicle'], 
             key_on='feature.properties.UBOS_PN_13', 
             fill_color='YlGn',
             fill_opacity=0.2, 
             line_opacity=0.8,
          #   threshold_scale=scale,     
             legend_name='Pollution (%)', 
             smooth_factor=0,     
             highlight=True
                 )

   

#folium.LayerControl().add_to(m)

for index, location_info in kla_garage.iterrows():
    folium.Marker([location_info["latitude"], 
                   location_info["longitude"]], 
                  popup=location_info["Name"]
                  ).add_to(m)

m

/usr/local/lib/python3.7/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [ ]:
# Creating choropleth map
m = folium.Map(location=[y_map, x_map], zoom_start=12,tiles=None)
#folium.TileLayer('Stamen Terrain',name="Light Map",control=False).add_to(Kla_inc_map)
folium.TileLayer('Openstreetmap').add_to(m)
folium.LayerControl


SuburbName = folium.features.GeoJson(
    klageo_gdf,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['UBOS_PN_13'
 #               ,'UBOS_PN_13'
 #               ,'UBOS_PN_13'
 #               ,'  Yearly average taxable income of Female   '
 #               ,'  Yearly average taxable income of Male   '
               ],
        aliases=['Parish :: '
 #               ,'Division :: '
 #               ,'CITY: '
 #               ,'  Yearly average taxable income of Female   : '
 #               ,'  Yearly average taxable income of Male   : ' 
                 ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(SuburbName)
m.keep_in_front(SuburbName)

# Adding folium layer to the map
folium.LayerControl().add_to(m)


for index, location_info in kla_garage.iterrows():
    folium.Marker([location_info["latitude"], 
                   location_info["longitude"]], 
                  popup=location_info["Name"]
                  ).add_to(m)

m





In [ ]:

#df = df.to_crs(epsg = 4326)
kla_geo.crs = "epsg:4326"

mm = folium.Map(location=[y_map, x_map], zoom_start=12,tiles=None)
#folium.TileLayer('Stamen Terrain',name="Light Map",control=False).add_to(Kla_inc_map)
folium.TileLayer('Openstreetmap').add_to(mm)
folium.LayerControl

folium.Choropleth(geo_data=kla_geo,
                    name="choropleth",

                      fill_opacity=0.2,
                     line_opacity=0.8,
                     ).add_to(mm)


# Adding informative lables to the map using geoJson

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}





SuburbName = folium.features.GeoJson(
    klageo_gdf,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['UBOS_PN_13'
                ,'Vehicle'
 #               ,'UBOS_PN_13'
 #               ,'  Yearly average taxable income of Female   '
 #               ,'  Yearly average taxable income of Male   '
               ],
        aliases=['Parish :: '
                ,'Vehicle repaired :: '
 #               ,'CITY: '
 #               ,'  Yearly average taxable income of Female   : '
 #               ,'  Yearly average taxable income of Male   : ' 
                 ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
mm.add_child(SuburbName)
mm.keep_in_front(SuburbName)




for index, location_info in kla_garage.iterrows():
    folium.Marker([location_info["latitude"], 
                   location_info["longitude"]], 
                  popup=location_info["Name"]
                  ).add_to(mm)



mm